In [1]:
import numpy as np

In [2]:
np.__version__

'1.18.3'

In [3]:
# 此行式子為 2 的 3.4.5 次方 = array([8,16,32])
# scales = 2 ** np.arange(3,6)
# python2 之 xrange() 在這邊改成 list(range())
# list(range(8)) [0, 1, 2, 3, 4, 5, 6, 7]
# ratio_acchors.shpae[0] 為 ratio_acchors 第一維長度

In [4]:
def generate_anchors(base_size=16,ratios=[0.5,1,2],scales = 2**np.arange(3,6)):
    base_anchor = np.array([1,1,base_size,base_size])-1
# array([ 0,  0, 15, 15])
    ratio_anchors = _ratio_enum(base_anchor,ratios) 
# vstack 矩陣上下相合併
# for i in xrange(ratio_acchors.shpae[0])
    
    #ratio_anchors_shape = list(ratio_anchors.shpae[0])
    for i in range(len(ratios)):
        anchors=np.vstack([_scale_enum(ratio_anchors[i,:],scales)])
    
    return anchors
# 產生 Anchor 透過 ratio 來決定大小，並在一個 (0,0,15,15) 的空間做畫

In [5]:
def _whctrs(anchor):
    """
    返回 width，height，x center，y center for window
    """
    w = anchor[2] - anchor[0] + 1
    h = anchor[3] - anchor[1] + 1
    x_ctr = anchor[0] + 0.5 * ( w - 1 )
    y_ctr = anchor[1] + 0.5 * ( h - 1 )
    
    return w,h,x_ctr,y_ctr

In [6]:
# ws:[23 16 11] , hs:[12 16 22]
def _mkanchors(ws,hs,x_ctr,y_ctr):
    """
    將 width 和 height 輸入進函數並返回一個 Anchors 的 set
    """
    # newaxis 意思是將數組進行轉置
    ws = ws[:,np.newaxis]
    hs = hs[:,np.newaxis]
    
    anchors = np.hstack((x_ctr - 0.5 * (ws - 1),  # hstack,vstack : 合併數組
                         y_ctr - 0.5 * (hs - 1),  # anchor : [[-3.5 2 18.5 13]
                         x_ctr + 0.5 * (ws - 1),  #           [0 0 15 15]           
                         y_ctr + 0.5 * (hs - 1))) #           [2.5 -3 12.5 18]]
    return anchors

In [7]:
def _ratio_enum(anchor,ratios):
    """
    列舉一個 Anchor 的三種寬高比 1:2,1:1,2:1
    """
    # 返回 width 和 height 還有中心座標 w:16,h:16,x_ctr:7.5,y_ctr=7.5
    w,h,x_ctr,y_ctr = _whctrs(anchor)
    
    # size : 16*16=256 
    size = w * h
    
    # 256 / ratios[0.5,1,2]=[512,256,128]
    size_ratios = size / ratios
    
    # round()方法返回 x 四舍五入的數字，sqrt()方法返回數字 x 的平方根
    # ws:[23 16 11] hs:[12 16 22] 相互對應
    ws = np.round(np.sqrt(size_ratios))
    hs = np.round(ws * ratios)
    
    #給定一組寬高向量，輸出各個預測窗口
    anchors = _mkanchors(ws,hs,x_ctr,y_ctr)
    return anchors

In [8]:
def _scale_enum(anchor,scales):
    """
    列舉 Anchor 的三種尺度 128*128，256*256，512*512
    """
    w,h,x_ctr,y_ctr = _whctrs(anchor)
    
    #[128 256 512]
    ws = w * scales
    hs = h * scales
    
    #[[-56 -56 71 71] [-120 -120 135 135] [-248 -248 263 263]]
    anchors = _mkanchors(ws,hs,x_ctr,y_ctr)
    return anchors

In [9]:
if __name__ ==  '__main__':
    import time
    t = time.time()
    #產生 Anchor 窗口
    a = generate_anchors()
    #顯示時間
    print(time.time()-t)
    print(a)
    #from IPython import embed
    #embed()

0.0
[[ -36.  -80.   51.   95.]
 [ -80. -168.   95.  183.]
 [-168. -344.  183.  359.]]
